In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import requests
import matplotlib.pyplot as plt

url = "https://upload.wikimedia.org/wikipedia/en/4/42/Beatles_-_Abbey_Road.jpg"
response = requests.get(url, stream=True)
response.raise_for_status()

img = Image.open(response.raw).convert('RGB')

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(   
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img = transform(img).unsqueeze(0).to(device)

model_option1.eval()
model_option2.eval()

with torch.no_grad():
    output1 = model_option1(img)
    output2 = model_option2(img)

threshold = 0.5

fig, (ax1, ax2) = plt.subplots(ncols=2)

ax1.imshow(img.squeeze(0).cpu().numpy().transpose((1, 2, 0)))
ax1.set_title("Original Image")

for ax, output, option in zip((ax2, ax3), (output1, output2), ("Option 1", "Option 2")):
    boxes = output[0]['boxes'].cpu().numpy()

    ax.imshow(img.squeeze(0).cpu().numpy().transpose((1, 2, 0)))
    ax.set_title(option)

    for box in boxes:
        x1, y1, x2, y2 = box
        w, h = x2 - x1, y2 - y1
        rect = plt.Rectangle((x1, y1), w, h, linewidth=1,edgecolor='r', facecolor='none')
        ax.add_patch(rect)

plt.show()
